## Kaggle European Soccer Database Analysis
### Merge Ball Event Data into Match Table

The data are downloaded here: [Kaggle European Soccer Database](https://www.kaggle.com/hugomathien/soccer), and ball event data have been extracted to local disk in previous notebook.

### Table of Contents
#### 1. [Aggregate Ball Event Data to Construct Team's Match Stats Table](#1)
#### 2. [Merge Team's Match Stats to Match Table](#2)

In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sqlite3
import pickle
from tqdm import tqdm
import os
import bs4

sns.set_style('whitegrid')
sns.set_context('notebook')

print('All general modules are imported.')

All general modules are imported.


In [7]:
# Establish connection to sql database

# database file name
db_filename = 'database.sqlite'

# change database file directory as you wish
data_folder = os.path.join(os.path.pardir, os.path.pardir, os.path.pardir, 'data_source', 'kaggle')
db_filedir = os.path.join(data_folder, db_filename)

try:
    con=sqlite3.connect(db_filedir)
    print('Connection to the database is established.\n')
except Exception as e:
    print('Unable to establish the connection.')

cursor=con.execute("select name from sqlite_master where type='table'")
print('Following Tables are found in the database:')
for i in cursor.fetchall():
    print('{}'.format(i[0]))

Connection to the database is established.

Following Tables are found in the database:
sqlite_sequence
Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


In [8]:
# read player table
try:
    player = pd.read_sql_query('select * from Player',con)
    print('Successfully load player table from the database.')
except Exception as e:
    print('Unable to load player table from the database.')

# read match table
try:
    match = pd.read_sql_query('select * from Match',con)
    print('Successfully load match table from the database.')
except Exception as e:
    print('Unable to load match table from the database.')
    
# read league table
try:
    league=pd.read_sql_query('select * from League',con)
    print('Successfully load league table from the database.')
except Exception as e:
    print('Unable to load league table from the database.')

# read country table
try:
    country=pd.read_sql_query('select * from Country',con)
    print('Successfully load country table from the database.')
except Exception as e:
    print('Unable to load country table from the database.')

# read team table
try:
    team=pd.read_sql_query('select * from Team', con)
    print('Successfully load team table from the database.')
except Exception as e:
    print('Unable to load team table from the database.')

Successfully load player table from the database.
Successfully load match table from the database.
Successfully load league table from the database.
Successfully load country table from the database.
Successfully load team table from the database.


#### <a id='1'></a>1. Aggregate Ball Event Data to Construct Team's Match Stats Table
#### 1.1 No. of Goals in Each Match for Each Team

In [9]:
# Construct match_team table with goal stats
match_result = match[['match_api_id','home_team_api_id','away_team_api_id','home_team_goal','away_team_goal']]

# Home team
home_team = match_result[['match_api_id','home_team_api_id','home_team_goal']]
home_team.columns = ['match_api_id','team','goal']
home_team['Ishometeam'] = 1

# Away team
away_team = match_result[['match_api_id','away_team_api_id','away_team_goal']]
away_team.columns = ['match_api_id','team','goal']
away_team['Ishometeam'] = 0

# Glue home team and away team together
match_team=pd.concat([home_team,away_team], ignore_index=True)

C:\Users\ZAO_LI_XU\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\ZAO_LI_XU\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [10]:
print(match_team.info())
match_team.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51958 entries, 0 to 51957
Data columns (total 4 columns):
match_api_id    51958 non-null int64
team            51958 non-null int64
goal            51958 non-null int64
Ishometeam      51958 non-null int64
dtypes: int64(4)
memory usage: 1.6 MB
None


,match_api_id,team,goal,Ishometeam
0,492473,9987,1,1
1,492474,10000,0,1
2,492475,9984,0,1
3,492476,9991,5,1
4,492477,7947,1,1


#### 1.2 No. of Shotons in Each Match for Each Team
Load shoton event data that were extracted from match table in section 3.

#### NOTE

Shoton events only include: shots that are on target but not converted to goals. So next shoton will be also referred as shoton_no_goal.

In [11]:
try:
    shoton=pd.read_pickle(os.path.join(data_folder, 'shoton.data'))
    print('Successfully reload pre-saved shoton.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved shoton.data.')

Successfully reload pre-saved shoton.data from local disk.


In [12]:
print(shoton.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93755 entries, 0 to 93754
Data columns (total 10 columns):
elapsed         93755 non-null int64
elapsed_plus    5968 non-null float64
id              93755 non-null int64
latitude        42982 non-null float64
longitude       42982 non-null float64
match_api_id    93755 non-null int64
player1         92802 non-null float64
subtype         93737 non-null object
team            93684 non-null float64
type            93755 non-null object
dtypes: float64(5), int64(3), object(2)
memory usage: 7.2+ MB
None


Drop records with team=NaN. This is confirmed from 'shoton' column content in match table. When team is NaN, the record is marked with 'del=1', which means this record should be deleted.

In [13]:
shoton = shoton.dropna(subset=['team'])
print(shoton.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93684 entries, 0 to 93754
Data columns (total 10 columns):
elapsed         93684 non-null int64
elapsed_plus    5963 non-null float64
id              93684 non-null int64
latitude        42981 non-null float64
longitude       42981 non-null float64
match_api_id    93684 non-null int64
player1         92802 non-null float64
subtype         93667 non-null object
team            93684 non-null float64
type            93684 non-null object
dtypes: float64(5), int64(3), object(2)
memory usage: 7.9+ MB
None


Transform shoton events into numbers of shoton for teams

In [14]:
shoton_team = shoton[['match_api_id','team','id']].groupby(by=['match_api_id','team'],as_index=False).count()
shoton_team = shoton_team.rename(columns={'id': 'shoton_no_goal'})
shoton_team.head(5)

,match_api_id,team,shoton_no_goal
0,489042,10260.0,11
1,489042,10261.0,1
2,489043,8659.0,2
3,489043,9825.0,12
4,489044,8472.0,4


Note: shoton_team table will not include: matches with match id shown in shoton_team table, but teams don't have shotons in the matches. So next add these records with shoton_no_goal = 0.

In [15]:
# Find out all teams shown in matches with all match ids in shoton_team table
shoton_team = pd.merge(match_team[match_team.match_api_id.isin(shoton_team.match_api_id)], shoton_team,
                       how='left',on=['match_api_id','team'])[['match_api_id','team','shoton_no_goal']]
shoton_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16926 entries, 0 to 16925
Data columns (total 3 columns):
match_api_id      16926 non-null int64
team              16926 non-null object
shoton_no_goal    16673 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 528.9+ KB


In [16]:
# Replace shoton = NaN with 0
shoton_team = shoton_team.fillna(0)

print(shoton_team.info())
shoton_team.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16926 entries, 0 to 16925
Data columns (total 3 columns):
match_api_id      16926 non-null int64
team              16926 non-null int64
shoton_no_goal    16926 non-null float64
dtypes: float64(1), int64(2)
memory usage: 528.9 KB
None


,match_api_id,team,shoton_no_goal
0,489042,10260,11.0
1,489043,9825,12.0
2,489044,8472,4.0
3,489045,8654,5.0
4,489046,10252,5.0


#### 1.3 No. of Shotoffs in Each Match for Each Team
Load shotoff event data that were extracted from match table in section 3.

In [17]:
try:
    shotoff=pd.read_pickle(os.path.join(data_folder, 'shotoff.data'))
    print('Successfully reload pre-saved shotoff.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved shotoff.data.')

Successfully reload pre-saved shotoff.data from local disk.


In [18]:
print(shotoff.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95303 entries, 0 to 95302
Data columns (total 10 columns):
elapsed         95303 non-null int64
elapsed_plus    5972 non-null float64
id              95303 non-null int64
latitude        42206 non-null float64
longitude       42206 non-null float64
match_api_id    95303 non-null int64
player1         94659 non-null float64
subtype         95114 non-null object
team            95246 non-null float64
type            95303 non-null object
dtypes: float64(5), int64(3), object(2)
memory usage: 7.3+ MB
None


Also drop records with team = NaN.

In [19]:
shotoff = shotoff.dropna(subset=['team'])
print(shotoff.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95246 entries, 0 to 95302
Data columns (total 10 columns):
elapsed         95246 non-null int64
elapsed_plus    5970 non-null float64
id              95246 non-null int64
latitude        42206 non-null float64
longitude       42206 non-null float64
match_api_id    95246 non-null int64
player1         94659 non-null float64
subtype         95060 non-null object
team            95246 non-null float64
type            95246 non-null object
dtypes: float64(5), int64(3), object(2)
memory usage: 8.0+ MB
None


Transform shotoff events into numbers of shotoff for teams

In [20]:
shotoff_team = shotoff[['match_api_id','team','id']].groupby(by=['match_api_id','team'],as_index=False).count()
shotoff_team = shotoff_team.rename(columns={'id': 'shotoff'})
shotoff_team.head()

,match_api_id,team,shotoff
0,489042,10260.0,10
1,489042,10261.0,9
2,489043,8659.0,3
3,489043,9825.0,13
4,489044,8472.0,3


In [21]:
# Find out all teams shown in matches with all match ids in shotoff_team table
shotoff_team = pd.merge(match_team[match_team.match_api_id.isin(shotoff_team.match_api_id)], shotoff_team,
                        how='left',on=['match_api_id','team'])[['match_api_id','team','shotoff']]
shotoff_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16926 entries, 0 to 16925
Data columns (total 3 columns):
match_api_id    16926 non-null int64
team            16926 non-null object
shotoff         16731 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 528.9+ KB


Also add shotoff=0 for NaN.

In [22]:
# Replace shotoff = NaN with 0
shotoff_team = shotoff_team.fillna(0)

print(shotoff_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16926 entries, 0 to 16925
Data columns (total 3 columns):
match_api_id    16926 non-null int64
team            16926 non-null int64
shotoff         16926 non-null float64
dtypes: float64(1), int64(2)
memory usage: 528.9 KB
None


#### 1.4 No. of Cards in Each Match for Each Team
Load card event data that were extracted from match table in section 3.

In [23]:
try:
    card = pd.read_pickle(os.path.join(data_folder, 'card.data'))
    print('Successfully reload pre-saved card.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved card.data.')

Successfully reload pre-saved card.data from local disk.


In [24]:
print(card.info())

<class 'pandas.core.frame.DataFrame'>
Index: 62114 entries, 0 to 62113
Data columns (total 9 columns):
cardtype        61509 non-null object
elapsed         62114 non-null int64
elapsed_plus    3482 non-null float64
id              62114 non-null int64
match_api_id    62114 non-null int64
player1         61791 non-null float64
subtype         18864 non-null object
team            61794 non-null float64
type            62114 non-null object
dtypes: float64(3), int64(3), object(3)
memory usage: 4.7+ MB
None


Also drop records with team = NaN.

In [25]:
card = card.dropna(subset=['team'])
card.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61794 entries, 0 to 62113
Data columns (total 9 columns):
cardtype        61509 non-null object
elapsed         61794 non-null int64
elapsed_plus    3457 non-null float64
id              61794 non-null int64
match_api_id    61794 non-null int64
player1         61791 non-null float64
subtype         18789 non-null object
team            61794 non-null float64
type            61794 non-null object
dtypes: float64(3), int64(3), object(3)
memory usage: 4.7+ MB


Transform card events into numbers of card for teams

In [26]:
card_team = card[['match_api_id','team','id']].groupby(by=['match_api_id','team'],as_index=False).count()
card_team = card_team.rename(columns={'id': 'card'})
card_team.head()

,match_api_id,team,card
0,489042,10260.0,3
1,489044,8650.0,2
2,489045,8528.0,1
3,489045,8654.0,2
4,489046,8456.0,1


In [27]:
# Find out all teams shown in matches with all match ids in card_team table
card_team = pd.merge(match_team[match_team.match_api_id.isin(card_team.match_api_id)], card_team,
                     how='left',on=['match_api_id','team'])[['match_api_id','team','card']]
card_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27546 entries, 0 to 27545
Data columns (total 3 columns):
match_api_id    27546 non-null int64
team            27546 non-null object
card            25118 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 860.8+ KB


Also add card=0 for NaN.

In [28]:
# Replace card = NaN with 0
card_team = card_team.fillna(0)
print(card_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27546 entries, 0 to 27545
Data columns (total 3 columns):
match_api_id    27546 non-null int64
team            27546 non-null int64
card            27546 non-null float64
dtypes: float64(1), int64(2)
memory usage: 860.8 KB
None


#### 1.5 No. of Crosses in Each Match for Each Team
Load cross event data that were extracted from match table in section 3.

In [30]:
try:
    cross = pd.read_pickle(os.path.join(data_folder, 'cross.data'))
    print('Successfully reload pre-saved cross.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved cross.data.')

Successfully reload pre-saved cross.data from local disk.


In [31]:
print(cross.info())

<class 'pandas.core.frame.DataFrame'>
Index: 284775 entries, 0 to 284774
Data columns (total 8 columns):
elapsed         284775 non-null int64
elapsed_plus    16119 non-null float64
id              284775 non-null int64
match_api_id    284775 non-null int64
player1         271419 non-null float64
subtype         284775 non-null object
team            284594 non-null float64
type            284775 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 19.6+ MB
None


Also drop records with team = NaN.

In [32]:
cross = cross.dropna(subset=['team'])
cross.info()

<class 'pandas.core.frame.DataFrame'>
Index: 284594 entries, 0 to 284774
Data columns (total 8 columns):
elapsed         284594 non-null int64
elapsed_plus    16105 non-null float64
id              284594 non-null int64
match_api_id    284594 non-null int64
player1         271417 non-null float64
subtype         284594 non-null object
team            284594 non-null float64
type            284594 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 19.5+ MB


Transform cross events into numbers of cross for teams

In [33]:
cross_team = cross[['match_api_id','team','id']].groupby(by=['match_api_id','team'],as_index=False).count()
cross_team = cross_team.rename(columns={'id': 'cross'})
print(cross_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16921 entries, 0 to 16920
Data columns (total 3 columns):
match_api_id    16921 non-null int64
team            16921 non-null float64
cross           16921 non-null int64
dtypes: float64(1), int64(2)
memory usage: 528.8 KB
None


In [34]:
# Find out all teams shown in matches with all match ids in cross_team table
cross_team = pd.merge(match_team[match_team.match_api_id.isin(cross_team.match_api_id)], cross_team,
                      how='left',on=['match_api_id','team'])[['match_api_id','team','cross']]
cross_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16930 entries, 0 to 16929
Data columns (total 3 columns):
match_api_id    16930 non-null int64
team            16930 non-null object
cross           16921 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 529.1+ KB


Also add cross=0 for NaN.

In [35]:
cross_team = cross_team.fillna(0)
print(cross_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16930 entries, 0 to 16929
Data columns (total 3 columns):
match_api_id    16930 non-null int64
team            16930 non-null int64
cross           16930 non-null float64
dtypes: float64(1), int64(2)
memory usage: 529.1 KB
None


#### 1.6 No. of Foulcommits in Each Match for Each Team
Load foulcommit event data that were extracted from match table in section 3.

In [36]:
try:
    foulcommit = pd.read_pickle(os.path.join(data_folder, 'foulcommit.data'))
    print('Successfully reload pre-saved foulcommit.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved foulcommit.data.')

Successfully reload pre-saved foulcommit.data from local disk.


In [37]:
print(foulcommit.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219234 entries, 0 to 219233
Data columns (total 9 columns):
elapsed         219234 non-null int64
elapsed_plus    13132 non-null float64
id              219234 non-null int64
match_api_id    219234 non-null int64
player1         211405 non-null float64
player2         190897 non-null float64
subtype         116380 non-null object
team            218824 non-null float64
type            219234 non-null object
dtypes: float64(4), int64(3), object(2)
memory usage: 15.1+ MB
None


Also drop records with team = NaN.

In [38]:
foulcommit = foulcommit.dropna(subset=['team'])
foulcommit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218824 entries, 0 to 219233
Data columns (total 9 columns):
elapsed         218824 non-null int64
elapsed_plus    13101 non-null float64
id              218824 non-null int64
match_api_id    218824 non-null int64
player1         211404 non-null float64
player2         190896 non-null float64
subtype         116209 non-null object
team            218824 non-null float64
type            218824 non-null object
dtypes: float64(4), int64(3), object(2)
memory usage: 16.7+ MB


Transform foul commit events into numbers of foulcommit for teams

In [39]:
foulcommit_team = foulcommit[['match_api_id','team','id']].groupby(by=['match_api_id','team'],as_index=False).count()
foulcommit_team = foulcommit_team.rename(columns={'id': 'foulcommit'})
foulcommit_team.head()

,match_api_id,team,foulcommit
0,489042,10260.0,16
1,489042,10261.0,11
2,489043,8659.0,9
3,489043,9825.0,11
4,489044,8472.0,13


In [40]:
# Find out all teams shown in matches with all match ids in foulcommit_team table
foulcommit_team = pd.merge(match_team[match_team.match_api_id.isin(foulcommit.match_api_id)], foulcommit_team, how='left', on=['match_api_id','team'])[['match_api_id','team','foulcommit']]
foulcommit_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16930 entries, 0 to 16929
Data columns (total 3 columns):
match_api_id    16930 non-null int64
team            16930 non-null object
foulcommit      16927 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 529.1+ KB


Also add cross=0 for NaN.

In [41]:
foulcommit_team = foulcommit_team.fillna(0)
print(foulcommit_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16930 entries, 0 to 16929
Data columns (total 3 columns):
match_api_id    16930 non-null int64
team            16930 non-null int64
foulcommit      16930 non-null float64
dtypes: float64(1), int64(2)
memory usage: 529.1 KB
None


#### 1.7 No. of Corners in Each Match for Each Team
Load corner event data that were extracted from match table in section 3.

In [42]:
try:
    corner = pd.read_pickle(os.path.join(data_folder, 'corner.data'))
    print('Successfully reload pre-saved corner.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved corner.data.')

Successfully reload pre-saved corner.data from local disk.


In [43]:
print(corner.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87839 entries, 0 to 87838
Data columns (total 8 columns):
elapsed         87839 non-null int64
elapsed_plus    5603 non-null float64
id              87839 non-null int64
match_api_id    87839 non-null int64
player1         85376 non-null float64
subtype         87676 non-null object
team            87753 non-null float64
type            87839 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 5.4+ MB
None


Also drop records with team is NaN.

In [44]:
corner = corner.dropna(subset=['team'])
corner.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87753 entries, 0 to 87838
Data columns (total 8 columns):
elapsed         87753 non-null int64
elapsed_plus    5597 non-null float64
id              87753 non-null int64
match_api_id    87753 non-null int64
player1         85376 non-null float64
subtype         87600 non-null object
team            87753 non-null float64
type            87753 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 6.0+ MB


Transform corner events into number of corners for teams

In [45]:
corner_team = corner[['match_api_id','team','id']].groupby(by=['match_api_id','team'],as_index=False).count()
corner_team = corner_team.rename(columns={'id': 'corner'})
corner_team.head()

,match_api_id,team,corner
0,489042,10260.0,6
1,489042,10261.0,6
2,489043,8659.0,5
3,489043,9825.0,7
4,489044,8472.0,1


Delete 1 mistaken record:

In [46]:
# match id=530123 has 3 team records, delete team=8472
corner_team = corner_team[~((corner_team['match_api_id']==530123)&(corner_team['team']==8472))]
print(corner_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16615 entries, 0 to 16615
Data columns (total 3 columns):
match_api_id    16615 non-null int64
team            16615 non-null float64
corner          16615 non-null int64
dtypes: float64(1), int64(2)
memory usage: 519.2 KB
None


In [47]:
# Find out all teams shown in matches with all match ids in corner_team table
corner_team = pd.merge(match_team[match_team.match_api_id.isin(corner_team.match_api_id)],corner_team,how='left',on=['match_api_id','team'])[['match_api_id','team','corner']]
corner_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16928 entries, 0 to 16927
Data columns (total 3 columns):
match_api_id    16928 non-null int64
team            16928 non-null object
corner          16615 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 529.0+ KB


Also add corner=0 for NaN.

In [48]:
corner_team = corner_team.fillna(0)
print(corner_team.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16928 entries, 0 to 16927
Data columns (total 3 columns):
match_api_id    16928 non-null int64
team            16928 non-null int64
corner          16928 non-null float64
dtypes: float64(1), int64(2)
memory usage: 529.0 KB
None


#### 1.8 Possession in Each Match for Each Team
Load possession event data that were extracted from match table in section 3.

In [49]:
try:
    possession = pd.read_pickle(os.path.join(data_folder, 'possession.data'))
    print('Successfully reload pre-saved possession.data from local disk.')
except Exception as e:
    print('Unable to reload pre-saved possession.data.')

Successfully reload pre-saved possession.data from local disk.


In [50]:
possession.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34815 entries, 0 to 34814
Data columns (total 8 columns):
awaypos         34796 non-null float64
elapsed         34815 non-null int64
elapsed_plus    6320 non-null float64
homepos         34796 non-null float64
id              34815 non-null int64
match_api_id    34815 non-null int64
subtype         34815 non-null object
type            34815 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 2.1+ MB


Drop records with homepos/awaypos is NaN.

In [51]:
possession = possession.dropna(subset=['homepos'])
possession.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34796 entries, 0 to 34814
Data columns (total 8 columns):
awaypos         34796 non-null float64
elapsed         34796 non-null int64
elapsed_plus    6317 non-null float64
homepos         34796 non-null float64
id              34796 non-null int64
match_api_id    34796 non-null int64
subtype         34796 non-null object
type            34796 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 2.4+ MB


Transform possession events into average possession for teams

In [52]:
possession_team = possession[['match_api_id','awaypos','homepos']].groupby(by=['match_api_id'],as_index=False).mean()
possession_team.head()

,match_api_id,awaypos,homepos
0,489042,45.25,54.75
1,489043,35.75,64.25
2,489044,54.50,45.50
3,489045,50.25,49.75
4,489046,48.50,51.50


In [53]:
possession_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8419 entries, 0 to 8418
Data columns (total 3 columns):
match_api_id    8419 non-null int64
awaypos         8419 non-null float64
homepos         8419 non-null float64
dtypes: float64(2), int64(1)
memory usage: 263.1 KB


In [54]:
# Home team possession
home_pos = possession_team[['match_api_id','homepos']]
home_pos = pd.merge(home_team[home_team.match_api_id.isin(home_pos.match_api_id)], home_pos,
                    how='left',on=['match_api_id'])[['match_api_id', 'team', 'homepos']]
home_pos = home_pos.rename(index=str, columns={'homepos':'possession'})

# Away team possession
away_pos = possession_team[['match_api_id','awaypos']]
away_pos = pd.merge(away_team[away_team.match_api_id.isin(away_pos.match_api_id)], away_pos,
                    how='left',on=['match_api_id'])[['match_api_id', 'team', 'awaypos']]
away_pos = away_pos.rename(index=str, columns={'awaypos':'possession'})

# Glue home team and away team together
possession_team = pd.concat([home_pos, away_pos],ignore_index=True)

In [55]:
possession_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16838 entries, 0 to 16837
Data columns (total 3 columns):
match_api_id    16838 non-null int64
team            16838 non-null int64
possession      16838 non-null float64
dtypes: float64(1), int64(2)
memory usage: 394.7 KB


#### Add all together

In [56]:
final_df_1 = pd.merge(cross_team,card_team,how='inner',on=['match_api_id','team'])
final_df_2 = pd.merge(final_df_1,foulcommit_team,how='inner',on=['match_api_id','team'])
final_df_3 = pd.merge(final_df_2,shoton_team,how='inner',on=['match_api_id','team'])
final_df_4 = pd.merge(final_df_3,shotoff_team,how='inner',on=['match_api_id','team'])
final_df_5 = pd.merge(final_df_4,corner_team,how='inner',on=['match_api_id','team'])
final_df_6 = pd.merge(final_df_5,possession_team,how='inner',on=['match_api_id','team'])
final_df_7 = pd.merge(final_df_6, match_team, how='inner', on=['match_api_id', 'team'])
match_stats = final_df_7
match_stats = match_stats.drop('goal', axis=1)

print(match_stats.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16248 entries, 0 to 16247
Data columns (total 10 columns):
match_api_id      16248 non-null int64
team              16248 non-null int64
cross             16248 non-null float64
card              16248 non-null float64
foulcommit        16248 non-null float64
shoton_no_goal    16248 non-null float64
shotoff           16248 non-null float64
corner            16248 non-null float64
possession        16248 non-null float64
Ishometeam        16248 non-null int64
dtypes: float64(7), int64(3)
memory usage: 1.4 MB
None


#### <a id='2'></a>2. Merge Team's Match Stats to Match Table
#### 2.1 Add readbale details to match table
#### 2.1.1 Substitute contry_id with country name 

In [57]:
# Change column names for country table to be ready for merge to match table
country.columns = ['country_id', 'country']
country.head(2)

,country_id,country
0,1,Belgium
1,1729,England


In [58]:
# Merage the country table to match table
try:
    match = pd.merge(match, country, on='country_id', how='left')
    print('Successfully merge country table to match table.')
    print('The first 3 rows of the merged table is shown below:')
    print(match.head(3))
except Exception as e:
    print('Unable to merge country table to match table.')

Successfully merge country table to match table.
The first 3 rows of the merged table is shown below:
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal   ...     \
0        492473              9987              9993               1   ...      
1        492474             10000              9994               0   ...      
2        492475              9984              8635               0   ...      

    VCH   VCD   VCA   GBH   GBD   GBA   BSH   BSD   BSA  country  
0  1.65  3.40  4.50  1.78  3.25  4.00  1.73  3.40  4.20  Belgium  
1  2.00  3.25  3.25  1.85  3.25  3.75  1.91  3.25  3.60  Belgium  
2  2.35  3.25  2.65  2.50  3.20  2.50  2.30  3.20  2.75  Belgium  

[3 rows x 116 columns]


#### 2.1.2 Substitute league_id with league name

In [59]:
# Change column names for league table to be ready for merge to match table
league.columns = ['league_id', 'country_id','league']
league.head(2)

,league_id,country_id,league
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League


In [60]:
# Merage the league table to match table
try:
    match = pd.merge(match, league[['league_id','league']], on='league_id', how='left')
    print('Sucessfully merge league table to match table.')
    print('The first 3 rows of the merged table is shown below:')
    print(match.head(3))
except Exception as e:
    print('Unable to merge league table to match table.')

Sucessfully merge league table to match table.
The first 3 rows of the merged table is shown below:
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0        492473              9987              9993               1   
1        492474             10000              9994               0   
2        492475              9984              8635               0   

            ...             VCD   VCA   GBH   GBD   GBA   BSH   BSD   BSA  \
0           ...            3.40  4.50  1.78  3.25  4.00  1.73  3.40  4.20   
1           ...            3.25  3.25  1.85  3.25  3.75  1.91  3.25  3.60   
2           ...            3.25  2.65  2.50  3.20  2.50  2.30  3.20  2.75   

   country           

#### 2.1.3 Substitute home_team_api_id with home_team_name

In [61]:
# Create home_team table by copying team table
home_team = team

# Change column names for home_team table to be ready for merge to match table
home_team.columns = ['id','home_team_api_id','home_team_fifa_api_id','home_team','home_team_short_name']
home_team.head(2)

,id,home_team_api_id,home_team_fifa_api_id,home_team,home_team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC


In [62]:
# Merage the home_team table to match table
try:
    match = pd.merge(match, home_team[['home_team_api_id','home_team']], on='home_team_api_id', how='left')
    print('Successfully merge home_team table to match table.')
    print('The first 3 rows of the merged table is shown below:')
    print(match.head(3))
except Exception as e:
    print('Unable to merge home_team table to match table.')

Successfully merge home_team table to match table.
The first 3 rows of the merged table is shown below:
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0        492473              9987              9993               1   
1        492474             10000              9994               0   
2        492475              9984              8635               0   

         ...           VCA   GBH   GBD   GBA   BSH   BSD   BSA  country  \
0        ...          4.50  1.78  3.25  4.00  1.73  3.40  4.20  Belgium   
1        ...          3.25  1.85  3.25  3.75  1.91  3.25  3.60  Belgium   
2        ...          2.65  2.50  3.20  2.50  2.30  3.20  2.75  Belgium   

                   league

#### 2.1.4 Sustitute away_team_api_id with away_team_name

In [63]:
# Create away_team table by copying team table
away_team = team

# Change column names for away_team table to be ready for merge to match table
away_team.columns = ['id','away_team_api_id','away_team_fifa_api_id','away_team','away_team_short_name']
away_team.head(2)

,id,away_team_api_id,away_team_fifa_api_id,away_team,away_team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC


In [64]:
# Merage the home_team table to match table
try:
    match = pd.merge(match, away_team[['away_team_api_id','away_team']], on='away_team_api_id', how='left')
    print('Successfully merge away_team table to match table.')
    print('The first 3 rows of the merged table is shown below:')
    print(match.head(3))
except Exception as e:
    print('Unable to merge away_team table to match table.')

Successfully merge away_team table to match table.
The first 3 rows of the merged table is shown below:
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0        492473              9987              9993               1   
1        492474             10000              9994               0   
2        492475              9984              8635               0   

         ...          GBH   GBD   GBA   BSH   BSD   BSA  country  \
0        ...         1.78  3.25  4.00  1.73  3.40  4.20  Belgium   
1        ...         1.85  3.25  3.75  1.91  3.25  3.60  Belgium   
2        ...         2.50  3.20  2.50  2.30  3.20  2.75  Belgium   

                   league          home_team         

#### 2.2 Add team's match stats to match table
#### 2.2.1 Add home team match stats to match table 

In [65]:
# Subset for home team only
home_team_match_stats = match_stats[match_stats['Ishometeam']==1]

# Change column names for home_team table to be ready for merge to match table
home_team_match_stats.columns = ['match_api_id','home_team_api_id','home_team_cross','home_team_card','home_team_foulcommit','home_team_shoton_no_goal','home_team_shotoff','home_team_corner','home_team_possession','Ishometeam']

# Drop home_team_api_id column, since it will be harder to merge to match table with home_team_api_id
home_team_match_stats = home_team_match_stats.drop('home_team_api_id',axis=1)

# Merge home_team_match_stats to match table
try:
    match = pd.merge(match, home_team_match_stats[home_team_match_stats.columns[:-1]], on='match_api_id', how='left')
    print('Successfully merge home_team_match_stats table to match table.')
    print('The first 3 rows of the merge table is shown below:')
    print(match.head(3))
except Exception as e:
    print('Unable to merge home_team_match_stats table to match table')

Successfully merge home_team_match_stats table to match table.
The first 3 rows of the merge table is shown below:
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0        492473              9987              9993               1   
1        492474             10000              9994               0   
2        492475              9984              8635               0   

           ...                           league          home_team  \
0          ...           Belgium Jupiler League           KRC Genk   
1          ...           Belgium Jupiler League   SV Zulte-Waregem   
2          ...           Belgium Jupiler League  KSV Cercle Brugge   

          away_team  home_team_cro

#### 2.2.2 Add away team match stats to match table

In [66]:
# Subset for away team only
away_team_match_stats = match_stats[match_stats['Ishometeam']==0]

# Change column names for away_team table to be ready for merge to match table
away_team_match_stats.columns = ['match_api_id','away_team_api_id','away_team_cross','away_team_card','away_team_foulcommit','away_team_shoton_no_goal','away_team_shotoff','away_team_corner','away_team_possession','Ishometeam']

# Drop away_team_api_id column, since it will be harder to merge to match table with away_team_api_id
away_team_match_stats = away_team_match_stats.drop('away_team_api_id',axis=1)

# Merge away_team_match_stats to match table
try:
    match = pd.merge(match, away_team_match_stats[away_team_match_stats.columns[:-1]], on='match_api_id', how='left')
    print('Successfully merge away_team_match_stats table to match table.')
    print('The first 3 rows of the merge table is shown below:')
    print(match.head(3))
except Exception as e:
    print('Unable to merge away_team_match_stats to match table.')

Successfully merge away_team_match_stats table to match table.
The first 3 rows of the merge table is shown below:
   id  country_id  league_id     season  stage                 date  \
0   1           1          1  2008/2009      1  2008-08-17 00:00:00   
1   2           1          1  2008/2009      1  2008-08-16 00:00:00   
2   3           1          1  2008/2009      1  2008-08-16 00:00:00   

   match_api_id  home_team_api_id  away_team_api_id  home_team_goal  \
0        492473              9987              9993               1   
1        492474             10000              9994               0   
2        492475              9984              8635               0   

           ...           home_team_shotoff  home_team_corner  \
0          ...                         NaN               NaN   
1          ...                         NaN               NaN   
2          ...                         NaN               NaN   

   home_team_possession  away_team_cross  away_team_card  

In [67]:
# Print out final match table information
print(match.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25979 entries, 0 to 25978
Columns: 133 entries, id to away_team_possession
dtypes: float64(110), int64(9), object(14)
memory usage: 26.6+ MB
None


In [128]:
# Save the merged match table to local disk for future use
if not os.path.isfile(os.path.join(data_folder, 'match_stats.data')):
    print('Saving reconstructed match stats table to file...')
    try:
        with open(os.path.join(data_folder, 'match_stats.data'), 'wb') as f:
            pickle.dump(match, f)
        print('Reconstructed match stats table is saved.')
    except Exception as e:
        print('Unable to save reconstructed match stats table.')
else:
    print('Reconstructed match stats table is cached in file: match_stats.data.')

Reconstructed match stats table is cached in file: match_stats.data.
